In [1]:
import tensorflow
import keras
print(tensorflow.__version__)
print(keras.__version__)

2.4.1
2.4.3


In [2]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls "/content/drive/My Drive/DogsCats/dataset"

Mounted at /content/drive
single_prediction  test_set  training_set


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.4))

classifier.add(Conv2D(128, (3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.4))

classifier.add(Conv2D(256, (3,3), activation='relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Flatten())
classifier.add(Dropout(0.4))

classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/content/drive/My Drive/DogsCats/dataset/training_set',target_size=(64,64),batch_size=32,class_mode='binary')
test_set = train_datagen.flow_from_directory('/content/drive/My Drive/DogsCats/dataset/test_set',target_size=(64,64),batch_size=32,class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
print(training_set[0])
print(test_set)

(array([[[[0.7153066 , 0.69161695, 0.6200884 ],
         [0.6829833 , 0.67252326, 0.59866583],
         [0.7389036 , 0.73571074, 0.6665801 ],
         ...,
         [0.9960785 , 0.9960785 , 0.9960785 ],
         [0.99338526, 0.99338526, 0.99338526],
         [0.9921569 , 0.9921569 , 0.9921569 ]],

        [[0.7153066 , 0.69161695, 0.6200884 ],
         [0.6829833 , 0.67252326, 0.59866583],
         [0.7389036 , 0.73571074, 0.6665801 ],
         ...,
         [0.9960785 , 0.9960785 , 0.9960785 ],
         [0.99338526, 0.99338526, 0.99338526],
         [0.9921569 , 0.9921569 , 0.9921569 ]],

        [[0.7153066 , 0.69161695, 0.6200884 ],
         [0.6829833 , 0.67252326, 0.59866583],
         [0.7389036 , 0.73571074, 0.6665801 ],
         ...,
         [0.9960785 , 0.9960785 , 0.9960785 ],
         [0.99338526, 0.99338526, 0.99338526],
         [0.9921569 , 0.9921569 , 0.9921569 ]],

        ...,

        [[0.37647063, 0.07561032, 0.09223695],
         [0.37581825, 0.06470957, 0.08496978

In [17]:
from keras.callbacks import Callback

class SensitivitySpecificityCallback(Callback):
  
    def __init__(self, val_data):
       super().__init__()
       self.validation_data = val_data
       print('Inside Init')
  
    def on_epoch_end(self, epoch, logs=None):
      print('Inside Epoch End')
      x_test = self.validation_data[0]
      y_test = self.validation_data[1]
      # x_test, y_test = self.validation_data
      predictions = self.model.predict(x_test)
      y_test = np.argmax(y_test, axis=-1)
      predictions = np.argmax(predictions, axis=-1)
      c = confusion_matrix(y_test, predictions)
      print('Confusion matrix:\n', c)
      print('sensitivity', c[0, 0] / (c[0, 1] + c[0, 0]))
      print('specificity', c[1, 1] / (c[1, 1] + c[1, 0]))

In [18]:
classifier.fit(training_set,steps_per_epoch=250,epochs=10,shuffle=True,validation_data=test_set,validation_steps=62,callbacks=[SensitivitySpecificityCallback(test_set)])

Inside Init
Epoch 1/30
250/250 [==============================] - ETA: 0s - loss: 0.6847 - accuracy: 0.5466 

KeyboardInterrupt: ignored

In [19]:
classifier.save('DogCatCNN3.h5')

In [20]:
from keras.models import load_model

classifier = load_model('DogCatCNN3.h5')
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0

In [21]:
print(test_set.class_indices)
resultdic = training_set.class_indices

{'cats': 0, 'dogs': 1}


In [22]:
import numpy as np
from keras.preprocessing import image

def predictAnimal(num):
  test_image = image.load_img('/content/drive/My Drive/DogsCats/dataset/single_prediction/cat_or_dog_' + str(num) +  '.jpg',target_size=(64,64))
  test_image = image.img_to_array(test_image)
  test_image = np.expand_dims(test_image,axis=0)
  test_image = test_image/255
  result = classifier.predict(test_image)
  print(result)
  if round(result[0][0])==1:
    print('Dog')
  else:
    print('Cat')
      
for i in range(1,10):
  predictAnimal(i)

[[0.6471687]]
Dog
[[0.53703165]]
Dog
[[0.66295075]]
Dog
[[0.64282835]]
Dog
[[0.5834857]]
Dog
[[0.54927826]]
Dog
[[0.5770632]]
Dog
[[0.581863]]
Dog
[[0.5848579]]
Dog
